In [ ]:
# NB : JALANKAN INI DULU
# pip install opencv-python ultralytics pyserial 

In [1]:
from ultralytics import YOLO
import cv2 
import time
import serial

In [2]:
# load arduino
arduino = serial.Serial("COM7", 115200, timeout=1) 
time.sleep(3)

In [3]:
# load yolo model
model = YOLO("yolov8n.pt") 

In [ ]:
cap = cv2.VideoCapture(0)

# atur window agar bisa resize
cv2.namedWindow("camera", cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # prediksi
    results = model.predict(frame, imgsz=640, conf=0.5, verbose=False)

    # jika ada object yang terdeteksi maka kendalikan robotnya
    if results[0].boxes is not None and len(results[0].boxes) > 0:
        # get id dari obejct
        class_id = int(results[0].boxes.cls[0])

        # get name dari object
        class_name = model.names[class_id]

        # jika person/orang yang terdeteksi maka belok kanan
        if class_name == 'person':
            # kotak-in yang terdeteksi sebagai person
            x1, y1, x2, y2 = map(int, results[0].boxes[0].xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

            # kasih label di atas kotaknya
            label = f'{class_name} {float(results[0].boxes[0].conf[0]):.2f}'
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)


            arduino.write(b"2")
            time.sleep(1)  # belok kanan
            arduino.write(b"1")
            time.sleep(2)  # maju beberapa detik
            arduino.write(b"0")
            time.sleep(3)
    else:
        # menampilkan text 'tidak ada object yang terdeteksi' di window
        text = 'tidak ada object yang terdeteksi'
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.6
        thickness = 1
        color = (0, 0, 255)
        (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)
        frame_height, frame_width = frame.shape[:2]
        x = (frame_width - text_width) // 2
        y = (frame_height + text_height) // 2
        cv2.putText(frame, text, (x, y), font, font_scale, color, thickness)


    # menampilkan text 'tekan q untuk keluar' di window 
    exit_text = "tekan 'q' untuk keluar"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    thickness = 1
    color = (255, 255, 255)
    (text_width, text_height), _ = cv2.getTextSize(exit_text, font, font_scale, thickness)
    x = (frame.shape[1] - text_width) // 2
    y = frame.shape[0] - 20
    cv2.putText(frame, exit_text, (x, y), font, font_scale, color, thickness)

    # tampilkan window-nya
    cv2.imshow('camera', frame)

    # exit pencet q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()